# Trial 2
something different, maybe. Hope this works! :')

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np
import re
import math

In [2]:
# with open("../df.pkl", "rb") as f:
#     df = pickle.load(f)
df = pd.read_excel('../data/merged_set.xlsx')

## Tools testing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from yake import KeywordExtractor
# from keybert import KeyBERT
from transformers import pipeline

/mnt/Data Volume/Projects/Storion/Codebase/storion_aggregation_analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### YAKE (Looks good, but too many keywords)

In [ ]:
kw_extractor = KeywordExtractor()

In [ ]:
# # Used for testing purposes
# text = df.loc[df["event_id"] == "case_1"].iloc[0]["content"]
# keywords = kw_extractor.extract_keywords(text)
# keywords

In [ ]:
# kw_sum = 0
# kw_sum = sum([keyword[1] for keyword in keywords])
# kw_sum

### Keybert (Useless)

In [ ]:
# # Used for testing purposes
# model = KeyBERT(model="all-mpnet-base-v2")
# keywords_kb = model.extract_keywords(text)
# keywords_kb

### NER_xlm-roberta-large-finetuned-conll03-english

In [ ]:
fair_ner_pipe = pipeline("token-classification", model="FacebookAI/xlm-roberta-large-finetuned-conll03-english")

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
# # Used for testing purposes
# raw_entities = fair_ner_pipe(text)
# text

In [ ]:
# raw_entities

In [ ]:
def combine_same_entities(_text, _raw_entities):
    prev_segment = None
    entities = []
    for segment in _raw_entities:
        segment['word'] = segment['word'].replace('▁', ' ')
        original_entity_word = segment['word']
        entity_word = segment['word'].rstrip()
        segment['end'] = segment['end'] - (len(original_entity_word) - len(entity_word))
        if segment['word'].isspace():
            continue
        appended_to_prev_segment = False
        if prev_segment is not None and prev_segment['entity'] == segment['entity']:
            if prev_segment['end'] == segment['start']:
                entities[-1]['word'] += segment['word']
                appended_to_prev_segment = True
            elif _text[prev_segment['end']:segment['start']].isspace():
                entities[-1]['word'] += _text[prev_segment['end']:segment['start']] + segment['word']
                appended_to_prev_segment = True

            if appended_to_prev_segment:
                entities[-1]['end'] = segment['end']
                entities[-1]['score'] = (entities[-1]['score'] + segment['score'])/2

        if not appended_to_prev_segment:
            original_entity_word = entity_word
            entity_word = entity_word.lstrip()
            segment['start'] = segment['start'] + (len(original_entity_word) - len(entity_word))
            entities.append({
                'entity': segment['entity'],
                'word': entity_word,
                'score': segment['score'],
                'start': segment['start'],
                'end': segment['end']
            })
        entities[-1]['word'] = re.sub(r' +', ' ', entities[-1]['word']).strip()
        prev_segment = segment.copy()
    return entities

In [ ]:
# combine_same_entities(text, raw_entities)

## Approach

In [ ]:
from scripts.trial_2.text_filteration import ContentFilterer

In [ ]:
content_filter = ContentFilterer()

In [ ]:
# selected_df = df.iloc[:200]
# selected_df = selected_df._append(df.loc[df['event_id'] == 'case_1'])
# selected_df = selected_df._append(df.loc[df['event_id'] == 'case_2'])
# selected_df = selected_df._append(df.loc[df['event_id'] == 'case_3'])

In [ ]:
# selected_df.tail()

In [ ]:
# cleaned_contents = []

# # For cleaning all the contents
# # for idx, row in tqdm(df.iterrows(), desc="Filtering contents with CCF...", total=len(df)):
# #     cleaned_content = content_filter.filter_text(row['title'], row['content'])
# #     cleaned_contents.append(cleaned_content)

# # For cleaning of selected sample set
# for idx, row in tqdm(df.iterrows(), desc="Filtering contents with CCF...", total=len(df)):
#     cleaned_content = content_filter.filter_text(row['title'], row['content'])
#     cleaned_contents.append(cleaned_content)

Filtering contents with CCF...: 100%|██████████| 354/354 [01:42<00:00,  3.45it/s]


In [ ]:
# import pickle

# with open("cleaned_contents.pkl", "wb") as f:
#     pickle.dump(cleaned_contents, f)

In [ ]:
import pickle

with open("cleaned_contents.pkl", "rb") as f:
    cleaned_contents = pickle.load(f)

In [ ]:
# # Sample of keyword extraction & NER
# _sample_text = " ".join(cleaned_contents[19])
# print(_sample_text)
# sample_keywords = kw_extractor.extract_keywords(_sample_text)
# print(sample_keywords)
# sample_entities = combine_same_entities(_sample_text, fair_ner_pipe(_sample_text))
# sample_entities

In [ ]:
def filter_non_entity_keywords(_keywords, _entities):
    entity_list = set([entity['word'] for entity in _entities])
    _keywords = [keyword for keyword in _keywords if keyword[0] not in entity_list]
    return _keywords

In [ ]:
from difflib import SequenceMatcher
from collections import defaultdict

# Function to calculate similarity between two strings
def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Function to create a similarity matrix for all words
def create_similarity_matrix(primary_word_list, secondary_word_list):
    height = len(primary_word_list)
    width = len(secondary_word_list)
    matrix = [[0] * width for _ in range(height)]

    for i, word1 in enumerate(primary_word_list):
        for j, word2 in enumerate(secondary_word_list):
            if i != j:  # Avoid self-comparison
                matrix[i][j] = calculate_similarity(word1, word2)
    return matrix

# Function to group words based on similarity matrix
def group_similar_words(word_list, top_values_threshold=None):
    similarity_matrix = np.array(create_similarity_matrix(word_list, word_list))
    # Dynamic thresholding 
    top_n_values = len(similarity_matrix[similarity_matrix>top_values_threshold]) if top_values_threshold is not None else similarity_matrix.shape[0]//1
    top_n_values = 1 if top_n_values==0 else top_n_values
    top_n_values *= -1
    top_n_similarities = np.partition(similarity_matrix.flatten(), top_n_values)[top_n_values:]
    threshold = np.mean(top_n_similarities)
    print(f"Threshold set to {threshold}")
    groups = []
    visited = set()

    for i, word in enumerate(word_list):
        if word not in visited:
            group = set()
            for j, similarity in enumerate(similarity_matrix[i]):
                if similarity >= threshold:
                    group.add(word_list[j])
            group.add(word)  # Include the current word
            groups.append(list(group))
            visited.update(group)  # Mark all words in this group as visited

    return groups

In [ ]:
# sample_kw_groups = group_similar_words([keyword[0] for keyword in sample_keywords], top_values_threshold=0.3)
# sample_kw_groups

In [ ]:
# SequenceMatcher(None, sample_kw_groups[0][1], sample_entities[2]['word']).ratio()

In [ ]:
# print(sample_kw_groups[0][1])
# print(sample_entities[2]['word'])

In [ ]:
def identify_same_entities(entities, threshold=0.55):
    entity_word_list = [entity['word'] for entity in entities]
    entity_similarities = np.array(create_similarity_matrix(entity_word_list, entity_word_list))
    entity_groups = []
    group_id = 0
    entity_group_ids = [None] * len(entities)
    is_grouped = [False] * len(entities)
    for i, entity in enumerate(entities):
        if not is_grouped[i]:
            group = [entity]
            is_grouped[i] = True
            entity_group_ids[i] = group_id
            shorlisted_similarities = (entity_similarities[i, :] > threshold)
            for j, is_shorlisted in enumerate(shorlisted_similarities):
                if not is_shorlisted or is_grouped[j]:
                    continue
                if entity['entity'] == entities[j]['entity']:
                    group.append(entities[j])
                    is_grouped[j] = True
                    entity_group_ids[j] = group_id
            entity_groups.append(group)
            group_id += 1
    return entity_groups, entity_group_ids

In [ ]:
# sample_entity_groups, sample_entity_group_ids = identify_same_entities(entities=sample_entities, threshold=0.55)
# sample_entity_groups

In [ ]:
from dataclasses import dataclass

@dataclass
class Entity:
    name: str
    entity_type: str
    score: np.float64

In [ ]:
def link_entity_relevant_keyword_groups(keyword_groups, entity_groups, entity_group_ids):
  entity_keyword_group_ids = dict() # Initialize dictionary to store keyword group ids for each entity group
  max_keyword_group_length = max([len(keyword_group) for keyword_group in keyword_groups]) # Get maximum number of keywords in a group
  keyword_lengths = np.array([[len(keyword) for keyword in keyword_group] + ([-1]*(max_keyword_group_length-len(keyword_group))) for keyword_group in keyword_groups]) # Get lengths of all keywords in each group
  for group_id, entity_group in enumerate(entity_groups): # Iterate over each entity group
    entity_keyword_group_ids[group_id] = [] # Initialize list to store keyword group ids for the entity group
    entity_words = [entity['word'] for entity in entity_group] # Get all words in the entity group
    entity_word_lengths = [len(entity['word']) for entity in entity_group] # Get lengths of all words in the entity group
    compared_entity_word = entity_words[np.argmax(entity_word_lengths)] # Get the longest word in the entity group
    compared_entity_word_length = len(compared_entity_word) # Get the length of the longest word in the entity group
    # Threshold need to be non-linearly inversed propotional (the more length of the word, the less propotion of the length of the word as threshold, the threshold = length x propotion, the propotion should be between 0 and 1)
    default_proportion = 0.7 # For 3 words
    acceptable_keyword_length_threshold = compared_entity_word_length * (default_proportion / math.log(compared_entity_word_length + 1, 10)) # Non-linearly inversed propotional threshold
    acceptable_keyword_length_threshold = 1 if acceptable_keyword_length_threshold < 1 else acceptable_keyword_length_threshold # Minimum threshold to 1 (in case of very short words)
    acceptable_keyword_length = [compared_entity_word_length - acceptable_keyword_length_threshold, compared_entity_word_length + acceptable_keyword_length_threshold] # Acceptable length range for keywords with the threshold
    shortlisted_keyword_groups = [any(group_checks) for group_checks in np.bitwise_and(keyword_lengths >= acceptable_keyword_length[0], keyword_lengths <= acceptable_keyword_length[1])] # Shortlist groups with atleast one keyword in the acceptable length range
    for i, is_shortlisted in enumerate(shortlisted_keyword_groups): # Iterate over shortlisted keyword groups
        # Exit if the keyword group is not shortlisted
        if not is_shortlisted:
            continue
        # Check if the entity group is relevant to the keyword group
        similarity_scores = [SequenceMatcher(None, entity_word, keyword).ratio() for entity_word in entity_words for keyword in keyword_groups[i]]
        if max(similarity_scores) == 1 or np.mean(similarity_scores) > 0.8:
            entity_keyword_group_ids[group_id].append(i)
    entity_keyword_group_ids[group_id] = None if len(entity_keyword_group_ids[group_id]) == 0 else entity_keyword_group_ids[group_id] # Set to None if no keyword group is relevant to the entity group
  return entity_keyword_group_ids

In [ ]:
# entity_keyword_group_links = link_entity_relevant_keyword_groups(sample_kw_groups, sample_entity_groups, sample_entity_group_ids)
# sample_entity_group_word_list = [[entity['word'] for entity in entity_group] for entity_group in sample_entity_groups]
# { max(sample_entity_group_word_list[entity_group_id], key=len): None if relevent_kw_group_ids is None else [sample_kw_groups[kw_group_id] for kw_group_id in relevent_kw_group_ids] for entity_group_id, relevent_kw_group_ids in entity_keyword_group_links.items()}

In [ ]:
# sample_entity_group_word_list

In [ ]:
# max(sample_entity_group_word_list[0], key=len)

In [ ]:
# sample_kw_groups

In [ ]:
# # For the moment flatten the ner and keyword groups by length
# proceeding_entity_list = [max(entity_group, key=len) for entity_group in sample_entity_group_word_list]
# proceeding_keyword_list = [max(keyword_group, key=len) for keyword_group in sample_kw_groups]
# proceeding_keyword_list = list(set(proceeding_keyword_list))

In [ ]:
# proceeding_keyword_list

### Article Grouping

In [ ]:
class Entity:
    entity: str
    word: str
    score: float
    start: int
    end: int

    def __init__(self, _mapping: dict):
        self.entity = _mapping['entity']
        self.word = _mapping['word']
        self.score = _mapping['score']
        self.start = _mapping['start']
        self.end = _mapping['end']

In [ ]:
# @dataclass
# class GroupKeyword:
#     name: str
#     threshold: float
#     existence_rate: float

# @dataclass
# class GroupEntity(GroupKeyword):
#     entity_type: str

@dataclass
class WordItem:
  name: str
  threshold: float
  importance_score: float  # Track importance
  frequency: int = 1  # Track occurrences

@dataclass
class GroupKeyword:
  existence_rate: float
  items: list[WordItem]

@dataclass
class GroupEntity(GroupKeyword):
  entity_type: str


In [ ]:
# Article group utility functions
def get_entity_similarity_matrix(primary_entity_list: list[GroupEntity], secondary_entity_list: list[Entity], top_n: int = None) -> np.ndarray:
  """Calculates the similarity between the group entity list and article entity list.

  Args:
      primary_entity_list (list[GroupEntity]): The list of group entities.
      secondary_entity_list (list[Entity]): The list of article entities.
      top_n (int, optional): The number of top entities(per entity group) to consider. Defaults to None (the square root of the no. of entities per group).

  Returns:
      np.ndarray: A matrix representing entity similarity scores.
  """
  # Get the top N count list to consider
  if top_n is not None:
    assert top_n > 0, "Top N must be greater than 0"
    assert all([top_n > len(entity_list.items) for entity_list in primary_entity_list]), "Top N must be greater than the number of entities in every group"
    top_n_list = [top_n] * len(primary_entity_list)
  else:
    top_n_list = [int(np.sqrt(len(entity_list.items))) for entity_list in primary_entity_list]
  
  # Create the similarity matrix
  group_similarity_matrices = []
  for i, entity_list in enumerate(primary_entity_list):
    top_n_entities = sorted(entity_list.items, key=lambda x: x.importance_score, reverse=True)[:top_n_list[i]]
    group_similarity_matrix = np.array(create_similarity_matrix(
      [entity.name for entity in top_n_entities],
      [entity.word for entity in secondary_entity_list]
    ))
    group_similarity_matrix = np.mean(group_similarity_matrix, axis=0)
    group_similarity_matrices.append(group_similarity_matrix)
  group_similarity_matrices = np.array(group_similarity_matrices)
  # Create the entity type match matrix
  type_match_matrix = np.array([[1 if entity.entity_type == entity2.entity else 0 for entity2 in secondary_entity_list] for entity in primary_entity_list])
  # Match the entity type matrix with the similarity matrix
  assert group_similarity_matrices.shape == type_match_matrix.shape # Ensure the shape of both matrices are the same
  final_score_matrix = group_similarity_matrices * type_match_matrix
  return final_score_matrix

def get_entity_type_matrix(primary_entity_list: list[GroupEntity], secondary_entity_list: list[Entity], entity_similarity_matrix: np.ndarray) -> np.ndarray:
  no_of_article_entities = len(secondary_entity_list)
  entity_type_matrix = []
  for i, entity_group in enumerate(primary_entity_list):
    try:
        max_index = np.argmax(entity_similarity_matrix[i])
    except Exception:
        print(entity_similarity_matrix)
        raise
    weighted_threshold = np.average([entity.threshold for entity in entity_group.items], weights=[entity.importance_score for entity in entity_group.items])
    if entity_similarity_matrix[i, max_index] < weighted_threshold:
      entity_type_matrix.append([None] * no_of_article_entities)
    else:
      entity_type_matrix.append([None] * max_index + [entity_group.entity_type] + [None] * (no_of_article_entities - max_index - 1))
  entity_type_matrix = np.array(entity_type_matrix)
  return entity_type_matrix

def get_kw_group_match_matrix(primary_kw_group_list: list[GroupKeyword], secondary_kw_list: list[str], top_n: int = None, match_threshold: float = 0.5) -> np.ndarray:
    """Calculates the match matrix between group keywords and article keywords.

    Args:
        primary_kw_group_list (list[GroupKeyword]): The list of group keywords.
        secondary_kw_list (list[str]): The list of article keywords.
        top_n (int, optional): The number of top keywords(per keyword group) to consider. 
            Defaults to None (the square root of the no. of keywords per group).
        match_threshold (float, optional): The minimum proportion of matches required 
            among top_n keywords. Defaults to 0.5.

    Returns:
        np.ndarray: A matrix where each cell represents either 0 (no match) or 
                   a match rate (if matched) for a keyword group.
    """
    # Determine top_n for each keyword group
    if top_n is not None:
        assert top_n > 0, "Top N must be greater than 0"
        assert all([top_n <= len(keyword_group.items) for keyword_group in primary_kw_group_list
        ]), "Top N must be less than or equal to the number of keywords in every group"
        top_n_list = [top_n] * len(primary_kw_group_list)
    else:
        top_n_list = [max(2, int(np.sqrt(len(keyword_group.items)))) for keyword_group in primary_kw_group_list]

    # Create the match matrix
    group_match_matrices = []
    
    for i, keyword_group in enumerate(primary_kw_group_list):
        # Sort by importance_score instead of context_score
        top_n_keywords = sorted(keyword_group.items, key=lambda x: x.importance_score, reverse=True)[:top_n_list[i]]
        
        # Create a matrix of binary matches (0 or 1) based on thresholds
        keyword_matches = []
        for keyword in top_n_keywords:
            # Check if any secondary keyword matches this word item based on its threshold
            match_vector = np.zeros(len(secondary_kw_list))
            
            for j, sec_keyword in enumerate(secondary_kw_list):
                # Use exact match or implement a similarity function here
                similarity = calculate_similarity(keyword.name, sec_keyword)
                if similarity >= keyword.threshold:
                    match_vector[j] = 1
                    
            keyword_matches.append(match_vector)
        
        keyword_matches = np.array(keyword_matches)
        
        # Calculate match rate for this group across all secondary keywords
        if len(keyword_matches) > 0:
            # Method 1: For each secondary keyword, check if enough top keywords are matched
            match_counts = np.sum(keyword_matches, axis=0)
            min_matches_needed = max(1, int(len(top_n_keywords) * match_threshold))
            
            # Binary decision for each secondary keyword
            group_match_vector = np.zeros(len(secondary_kw_list))
            for j in range(len(secondary_kw_list)):
                if match_counts[j] >= min_matches_needed:
                    # Calculate match rate: weighted by importance and existence rate
                    matched_keywords = [top_n_keywords[k] for k in range(len(top_n_keywords)) if keyword_matches[k, j] == 1]
                    
                    # Method for match rate: weighted average of matched keywords' importance
                    total_importance = sum(kw.importance_score for kw in matched_keywords)
                    max_possible = sum(kw.importance_score for kw in top_n_keywords[:min_matches_needed])
                    
                    # Scale by the group's existence rate
                    match_rate = (total_importance / max_possible) * keyword_group.existence_rate
                    
                    group_match_vector[j] = match_rate
        else:
            group_match_vector = np.zeros(len(secondary_kw_list))
            
        group_match_matrices.append(group_match_vector)
    
    return np.array(group_match_matrices)

def calculate_threshold(phrase, base=0.7, len_penalty=0.01, token_penalty=0.05):
  """Calculates the threshold for a given phrase.

  Args:
      phrase (str): The phrase to calculate the threshold for.
      base (float, optional): The base threshold. Defaults to 0.7.
      len_penalty (float, optional): The length penalty. Defaults to 0.01.
      token_penalty (float, optional): The token penalty. Defaults to 0.05.

  Returns:
      float: The calculated threshold
  """
  tokens = phrase.split()
  length = len(phrase)
  num_tokens = len(tokens)
  threshold = base - (len_penalty * length) - (token_penalty * (num_tokens - 1))
  return max(0.2, min(0.9, threshold))  # Clamp between 0.2 and 0.9

def calculate_importance_score(*, initial: bool = False, total_articles: int | None = None, similarity_score: float | None = None, importance_score: float | None = None, frequency: int = 1) -> float:
  """Calculates the importance score for a given article.

  Args:
      initial (bool, optional): Indicates if this is the initial calculation. Defaults to False.
      total_articles (int | None, optional): The total number of articles. Defaults to None.
      similarity_score (float | None, optional): The similarity score related to the article. Defaults to None.
      importance_score (float | None, optional): The current importance score. Defaults to None.
      frequency (int, optional): The frequency of the article's appearance. Defaults to 1.

  Raises:
      ValueError: If total_articles is not provided.
      ValueError: If similarity_score is not provided for initial importance score calculation.
      ValueError: If importance_score is not provided for importance score calculation.
      ValueError: If frequency is not provided for initial importance score calculation.
      ValueError: If frequency is greater than total articles.

  Returns:
      float: The calculated importance score
  """
  if total_articles is None:
      raise ValueError("Total articles must be provided")
  
  if initial:
    if similarity_score is None:
        raise ValueError("Similarity score must be provided for initial importance score calculation")
  else:
    if importance_score is None:
        raise ValueError("Importance score must be provided for importance score calculation")
  
  if frequency is None:
      raise ValueError("Frequency must be provided for initial importance score calculation")
  elif frequency > total_articles:
      raise ValueError("Frequency must be less than total articles")
  
  if initial:
      return similarity_score * (frequency / total_articles)
  else:
      # Update the importance with the new article addition (use importance_score, frequency and total_articles only)
      return importance_score + (frequency / total_articles)

In [ ]:
entity_weights = {
  "I-PER": 0.4, # Person
  "I-LOC": 0.5, # Location
  "I-ORG": 0.3, # Organization
  "I-MISC": 0.2 # Miscellaneous
}

class ArticleGroup:
  identifier: int
  __internal_entities: list[GroupEntity]
  __internal_keywords: list[GroupKeyword]
  __group_article_ids: list[str]
  __total_entity_scorable: float

  def __init__(self, identifier: int = -1):
    self.identifier = identifier
    self.__group_article_ids = []
    self.__internal_entities = []
    self.__total_entity_scorable = 0
    self.__internal_keywords = []

  def get_group_article_ids(self):
    return self.__group_article_ids
  
  def get_group_keywords(self):
    return self.__internal_keywords
  
  def get_group_entities(self):
    return self.__internal_entities
  
  def __add_article_id(self, article_id: str):
    self.__group_article_ids.append(article_id)
  
  def add_initial_article(self, article_id: str, entities: list[dict], keywords: list[str]):
    """
    Add the first article to an empty article group, initializing its entities and keywords.

    Args:
        article_id (str): Unique identifier for the article
        entities (list[dict]): List of entities with 'word' and 'entity' keys
        keywords (list[str]): List of keywords from the article
    """
    global entity_weights

    # Initialize entities
    for entity in entities:
        # Create a new entity group for each unique entity type
        entity_group = GroupEntity(
            items=[],
            entity_type=entity.entity,
            existence_rate=1.0  # First article, so full existence rate
        )
        
        # Create a word item for the entity
        word_item = WordItem(
            name=entity.word.lower(),
            frequency=1,
            threshold=calculate_threshold(entity.word),
            importance_score=1.0  # Maximum initial importance
        )
        
        # Add word item to the entity group
        entity_group.items.append(word_item)
        
        # Add entity group to internal entities
        self.__internal_entities.append(entity_group)

    # Calculate total entity scorable
    self.__total_entity_scorable = sum([entity_weights[entity.entity] for entity in entities])

    # Initialize keywords
    for keyword in keywords:
        # Create a new keyword group
        keyword_group = GroupKeyword(
            items=[],
            existence_rate=1.0  # First article, so full existence rate
        )
        
        # Create a word item for the keyword
        word_item = WordItem(
            name=keyword.lower(),
            frequency=1,
            threshold=calculate_threshold(keyword),
            importance_score=1.0  # Maximum initial importance
        )
        
        # Add word item to the keyword group
        keyword_group.items.append(word_item)
        
        # Add keyword group to internal keywords
        self.__internal_keywords.append(keyword_group)

    # Set total keyword scorable
    self.__total_keyword_scorable = len(keywords)

    # Add the article ID to the group
    self.__add_article_id(article_id)

#   def add_article(self, article_id: str, entities: list[Entity], keywords: list[str]):
#     # If this is the first article in the group, use add_initial_article
#     if len(self.__group_article_ids) == 0:
#         self.add_initial_article(article_id, entities, keywords)
#         return

#     global entity_weights
#     no_of_articles = len(self.__group_article_ids)
#     # Mutate the list of entities with the new entities
#     ## Get entity similarity score matrix
#     entity_similarity_matrix = get_entity_similarity_matrix(self.__internal_entities, entities)
#     ## Create entity type matrix
#     entity_type_matrix = get_entity_type_matrix(self.__internal_entities, entities, entity_similarity_matrix)
#     ## Get Matched/Non-existent status of entities
#     is_matched = np.any(entity_type_matrix, axis=1)
#     for i, entity_type_matrix_col in enumerate(entity_type_matrix.T):
#       ## Update matched entity groups
#       if is_matched[i]:
#         matched_entity_group_index = np.argmax(entity_type_matrix[:, i] is not None)
#         ### Check if the entity word already exists
#         found_match = False
#         for existing_entity in self.__internal_entities[matched_entity_group_index].items:
#           if existing_entity.name == entities[i].word.lower():
#             existing_entity.frequency += 1
#             existing_entity.importance_score=calculate_importance_score(
#                 initial=False,
#                 importance_score=existing_entity.importance_score,
#                 frequency=existing_entity.frequency,
#                 total_articles=no_of_articles
#             )
#             found_match = True
#             break
#         if not found_match:
#           entity = WordItem(
#             name=entities[i].word,
#             threshold=calculate_threshold(entities[i].word),
#             importance_score=calculate_importance_score(
#                 initial=True,
#                 similarity_score=entity_similarity_matrix[matched_entity_group_index, i], 
#                 frequency=1,
#                 total_articles=no_of_articles
#             )
#           )
#           self.__internal_entities[matched_entity_group_index].items.append(entity)
#     ## Add non-existent entities
#     ## Pick non-existent entities (to a new entity group) with the help of the entity type matrix
#     for i, entity_type_matrix_col in enumerate(entity_type_matrix.T):
#       if not any(entity_type_matrix_col):
#         entity_group = GroupEntity(
#           entity_type=entities[i].entity,
#           existence_rate=0,  # Calculated later
#           items=[]
#         )
#         entity = WordItem(
#           name=entities[i].word,
#           frequency=1,
#           threshold=calculate_threshold(entities[i].word),
#           importance_score=1
#         )
#         entity_group.items.append(entity)
#         self.__internal_entities.append(entity_group)
#     ## Re-calculate existence rate of the entities
#     new_no_of_articles = no_of_articles + 1
#     is_updated = np.any(entity_type_matrix is not None).tolist()
#     is_updated = [is_updated] if isinstance(is_updated, bool) else is_updated
#     no_of_added_groups = len(self.__internal_entities) - len(is_updated)
#     is_updated.extend([True]*no_of_added_groups)
#     for entity_group, is_modded in zip(self.__internal_entities, is_updated):
#         eq_constant = 1 if is_modded else 0
#         entity_group.existence_rate = ((entity_group.existence_rate * no_of_articles) + eq_constant) / new_no_of_articles
      
#     ## Total entity scorable calculation
#     self.__total_entity_scorable = sum([entity_weights[entity.entity] for entity in entities])
    
#     # Mutate the list of keywords with the new keywords
#     ## Get keyword match matrix
#     kw_match_matrix = get_kw_group_match_matrix(self.__internal_keywords, keywords)
    
#     ## Update matched keyword groups
#     for i in range(len(keywords)):
#         # For each keyword, find all groups that matched with it
#         matched_groups = []
#         for j in range(len(self.__internal_keywords)):
#             if kw_match_matrix[j, i] > 0:
#                 matched_groups.append((j, kw_match_matrix[j, i]))  # Store (group_index, match_rate)
        
#         if matched_groups:
#             # Sort by match rate to find the best matching group
#             matched_groups.sort(key=lambda x: x[1], reverse=True)
#             best_match_group_index = matched_groups[0][0]
            
#             ### Check if the keyword already exists in the best matching group
#             found_match = False
#             for existing_keyword in self.__internal_keywords[best_match_group_index].items:
#                 if existing_keyword.name.lower() == keywords[i].lower():
#                     existing_keyword.frequency += 1
#                     existing_keyword.importance_score = calculate_importance_score(
#                         initial=False,
#                         importance_score=existing_keyword.importance_score,
#                         frequency=existing_keyword.frequency,
#                         total_articles=no_of_articles
#                     )
#                     found_match = True
#                     break
            
#             if not found_match:
#                 # Add the keyword to the best matching group
#                 keyword = WordItem(
#                     name=keywords[i],
#                     threshold=calculate_threshold(keywords[i]),
#                     importance_score=calculate_importance_score(
#                         initial=True,
#                         similarity_score=matched_groups[0][1],  # Use match rate as similarity score
#                         frequency=1,
#                         total_articles=no_of_articles
#                     )
#               )
#                 self.__internal_keywords[best_match_group_index].items.append(keyword)
    
#     # Get indices of keywords that didn't match any group
#     non_matched_keywords = [i for i in range(len(keywords)) if not any(kw_match_matrix[:, i] > 0)]
    
#     # Create new keyword groups for non-matched keywords
#     for i in non_matched_keywords:
#         keyword_group = GroupKeyword(
#             items=[],
#             existence_rate=0  # Will be calculated later
#         )
#         keyword = WordItem(
#             name=keywords[i],
#             frequency=1,
#             threshold=calculate_threshold(keywords[i]),
#             importance_score=1  # Initial importance score
#         )
#         keyword_group.items.append(keyword)
#         self.__internal_keywords.append(keyword_group)
  
#     ## Re-calculate existence rate of the keywords
#     new_no_of_articles = no_of_articles + 1
#     is_kw_updated = np.any(kw_match_matrix > 0, axis=1).tolist()
#     is_kw_updated = [is_kw_updated] if isinstance(is_kw_updated, bool) else is_kw_updated
#     no_of_added_kw_groups = len(self.__internal_keywords) - len(is_kw_updated)
#     is_kw_updated.extend([True] * no_of_added_kw_groups)
    
#     for keyword_group, is_modded in zip(self.__internal_keywords, is_kw_updated):
#         eq_constant = 1 if is_modded else 0
#         keyword_group.existence_rate = ((keyword_group.existence_rate * no_of_articles) + eq_constant) / new_no_of_articles

#     ## Total keyword scorable calculation - similar concept to entity scorable
#     self.__total_keyword_scorable = len(keywords)  # Simple count, could be weighted if needed
    
#     # Add the article ID to the group
#     self.__add_article_id(article_id)

  def add_article(self, article_id: str, entities: list[Entity], keywords: list[str]):
    # Check if duplicate article
    if article_id in self.__group_article_ids:
        print(f"Article {article_id} already exists in the group.")
        return
    
    # If this is the first article in the group, use add_initial_article
    if len(self.__group_article_ids) == 0:
        self.add_initial_article(article_id, entities, keywords)
        return

    global entity_weights
    self.__add_article_id(article_id)
    no_of_articles = len(self.__group_article_ids)
    # Mutate the list of entities with the new entities
    ## Get entity similarity score matrix
    entity_similarity_matrix = get_entity_similarity_matrix(self.__internal_entities, entities)
    ## Create entity type matrix
    entity_type_matrix = get_entity_type_matrix(self.__internal_entities, entities, entity_similarity_matrix)
    ## Get Matched/Non-existent status of entities
    is_matched = np.any(entity_type_matrix, axis=0)
    
    # Track processed entities to avoid duplicate frequency increments
    processed_entities = set()
    
    for i, entity_type_matrix_col in enumerate(entity_type_matrix.T):
      ## Update matched entity groups
      if is_matched[i]:
        matched_entity_group_index = np.argmax(entity_type_matrix[:, i] is not None)
        entity_word = entities[i].word.lower()
        
        # Create a unique key for this entity in this group
        entity_key = f"{matched_entity_group_index}:{entity_word}"
        
        ### Check if the entity word already exists
        found_match = False
        
        for existing_entity in self.__internal_entities[matched_entity_group_index].items:
          if existing_entity.name == entity_word:
            # Only increment frequency if this entity hasn't been processed yet for this article
            if entity_key not in processed_entities:
              existing_entity.frequency += 1
              existing_entity.importance_score=calculate_importance_score(
                  initial=False,
                  importance_score=existing_entity.importance_score,
                  frequency=existing_entity.frequency,
                  total_articles=no_of_articles
              )
              # Mark this entity as processed for this article
              processed_entities.add(entity_key)
            found_match = True
            break
            
        if not found_match:
          entity = WordItem(
            name=entities[i].word,
            threshold=calculate_threshold(entities[i].word),
            importance_score=calculate_importance_score(
                initial=True,
                similarity_score=entity_similarity_matrix[matched_entity_group_index, i], 
                frequency=1,
                total_articles=no_of_articles
            )
          )
          self.__internal_entities[matched_entity_group_index].items.append(entity)
          # No need to add to processed_entities as we're creating a new entity with frequency 1
          
    ## Add non-existent entities
    ## Pick non-existent entities (to a new entity group) with the help of the entity type matrix
    for i, entity_type_matrix_col in enumerate(entity_type_matrix.T):
      if not any(entity_type_matrix_col):
        entity_group = GroupEntity(
          entity_type=entities[i].entity,
          existence_rate=0,  # Calculated later
          items=[]
        )
        entity = WordItem(
          name=entities[i].word,
          frequency=1,
          threshold=calculate_threshold(entities[i].word),
          importance_score=1
        )
        entity_group.items.append(entity)
        self.__internal_entities.append(entity_group)
        # No need to add to processed_entities as we're creating a new entity group with frequency 1
        
    ## Re-calculate existence rate of the entities
    new_no_of_articles = no_of_articles + 1
    is_updated = np.any(entity_type_matrix is not None).tolist()
    is_updated = [is_updated] if isinstance(is_updated, bool) else is_updated
    no_of_added_groups = len(self.__internal_entities) - len(is_updated)
    is_updated.extend([True]*no_of_added_groups)
    for entity_group, is_modded in zip(self.__internal_entities, is_updated):
        eq_constant = 1 if is_modded else 0
        entity_group.existence_rate = ((entity_group.existence_rate * no_of_articles) + eq_constant) / new_no_of_articles
      
    ## Total entity scorable calculation
    self.__total_entity_scorable = sum([entity_weights[entity.entity] for entity in entities])
    
    # Mutate the list of keywords with the new keywords
    ## Get keyword match matrix
    kw_match_matrix = get_kw_group_match_matrix(self.__internal_keywords, keywords)
    
    # Track processed keywords to avoid duplicate frequency increments
    processed_keywords = set()
    
    ## Update matched keyword groups
    for i in range(len(keywords)):
        keyword_lower = keywords[i].lower()
        
        # For each keyword, find all groups that matched with it
        matched_groups = []
        for j in range(len(self.__internal_keywords)):
            if kw_match_matrix[j, i] > 0:
                matched_groups.append((j, kw_match_matrix[j, i]))  # Store (group_index, match_rate)
        
        if matched_groups:
            # Sort by match rate to find the best matching group
            matched_groups.sort(key=lambda x: x[1], reverse=True)
            best_match_group_index = matched_groups[0][0]
            
            # Create a unique key for this keyword in this group
            keyword_key = f"{best_match_group_index}:{keyword_lower}"
            
            ### Check if the keyword already exists in the best matching group
            found_match = False
            for existing_keyword in self.__internal_keywords[best_match_group_index].items:
                if existing_keyword.name.lower() == keyword_lower:
                    # Only increment frequency if this keyword hasn't been processed yet for this article
                    if keyword_key not in processed_keywords:
                        existing_keyword.frequency += 1
                        try:
                          existing_keyword.importance_score = calculate_importance_score(
                              initial=False,
                              importance_score=existing_keyword.importance_score,
                              frequency=existing_keyword.frequency,
                              total_articles=no_of_articles
                          )
                        except Exception:
                          print(f"Error calculating importance score for keyword: {existing_keyword.name}")
                          print(f"Frequency: {existing_keyword.frequency}, Total Articles: {no_of_articles}")
                          raise
                        # Mark this keyword as processed for this article
                        processed_keywords.add(keyword_key)
                    found_match = True
                    break
            
            if not found_match:
                # Add the keyword to the best matching group
                keyword = WordItem(
                    name=keywords[i],
                    threshold=calculate_threshold(keywords[i]),
                    importance_score=calculate_importance_score(
                        initial=True,
                        similarity_score=matched_groups[0][1],  # Use match rate as similarity score
                        frequency=1,
                        total_articles=no_of_articles
                    )
                )
                self.__internal_keywords[best_match_group_index].items.append(keyword)
                # No need to add to processed_keywords as we're creating a new keyword with frequency 1
    
    # Get indices of keywords that didn't match any group
    non_matched_keywords = [i for i in range(len(keywords)) if not any(kw_match_matrix[:, i] > 0)]
    
    # Create new keyword groups for non-matched keywords
    for i in non_matched_keywords:
        keyword_group = GroupKeyword(
            items=[],
            existence_rate=0  # Will be calculated later
        )
        keyword = WordItem(
            name=keywords[i],
            frequency=1,
            threshold=calculate_threshold(keywords[i]),
            importance_score=1  # Initial importance score
        )
        keyword_group.items.append(keyword)
        self.__internal_keywords.append(keyword_group)
        # No need to add to processed_keywords as we're creating a new keyword group with frequency 1
  
    ## Re-calculate existence rate of the keywords
    new_no_of_articles = no_of_articles + 1
    is_kw_updated = np.any(kw_match_matrix > 0).tolist()
    is_kw_updated = [is_kw_updated] if isinstance(is_kw_updated, bool) else is_kw_updated
    no_of_added_kw_groups = len(self.__internal_keywords) - len(is_kw_updated)
    is_kw_updated.extend([True] * no_of_added_kw_groups)
    
    for keyword_group, is_modded in zip(self.__internal_keywords, is_kw_updated):
        eq_constant = 1 if is_modded else 0
        keyword_group.existence_rate = ((keyword_group.existence_rate * no_of_articles) + eq_constant) / new_no_of_articles

    ## Total keyword scorable calculation - similar concept to entity scorable
    self.__total_keyword_scorable = len(keywords)  # Simple count, could be weighted if needed
    
    # # Add the article ID to the group
    # self.__add_article_id(article_id)


  def get_group_relevance_score(self, entities: list[dict], keywords: list[str], 
                            entity_match_threshold: float = 0.8, 
                            keyword_match_threshold: float = 0.8, 
                            match_threshold: float = 0.7) -> bool:
    """Calculates the relevance score of the article group based on the entities and keywords.

    Args:
        entities (list[dict]): List of entity dictionaries with 'word' and 'entity' keys
        keywords (list[str]): List of keywords from the article
        entity_match_threshold (float, optional): Threshold for entity matching. Defaults to 0.8.
        keyword_match_threshold (float, optional): Threshold for keyword matching. Defaults to 0.8.
        match_threshold (float, optional): Overall match threshold. Defaults to 0.7.

    Returns:
        bool: Whether the article is relevant to the group
    """
    global entity_weights
    # Check entity relevance (40% weight)
    ## Get entity similarity score matrix
    entity_similarity_matrix = get_entity_similarity_matrix(self.__internal_entities, entities)
    ## Create entity type matrix
    try:
        entity_type_matrix = get_entity_type_matrix(
            self.__internal_entities, 
            entities, 
            entity_similarity_matrix
        )
    except Exception as e:
        print(f"Error creating entity type matrix: {e}")
        print(f"Internal Entities: {self.__internal_entities}")
        print(f"Entities: {entities}")
        print(f"Entity Similarity Matrix: {entity_similarity_matrix}")
        raise
    ## Calculate entity weight matrix
    entity_weight_matrix = np.array([
        [entity_weights.get(entity_type, 0) if entity_type is not None else 0 
        for entity_type in entity_type_matrix_row] 
        for entity_type_matrix_row in entity_type_matrix
    ])
    ## Calculate entity score
    # Sum of weights for matched entities, scaled by their similarity and type weights
    entity_score = np.sum(entity_weight_matrix)
    ## Normalize entity score (40% weight)
    # Prevent division by zero
    normalized_entity_score = (
        (entity_score / max(self.__total_entity_scorable, 1)) * 0.4 
        if self.__total_entity_scorable > 0 
        else 0
    )
    # Early exit if no entity match
    if normalized_entity_score == 0:
        return False
    # Check keyword relevance (60% weight)
    ## Use the new group match matrix
    kw_match_matrix = get_kw_group_match_matrix(self.__internal_keywords, keywords)
    # Calculate keyword score
    # Sum of match rates across all keyword groups
    keyword_score = np.sum(kw_match_matrix)
    # Normalize keyword score (60% weight)
    # Prevent division by zero and handle empty keywords
    normalized_keyword_score = (
        (keyword_score / max(self.__total_keyword_scorable, 1)) * 0.6 
        if self.__total_keyword_scorable > 0 and keywords 
        else 0
    )
    # Calculate total score
    total_score = normalized_entity_score + normalized_keyword_score
    # Return whether total score meets the match threshold
    return total_score >= match_threshold

In [ ]:
# Implement article grouping and export results to a csv with the article_id and group_id of which it was grouped 

def group_articles(threshold: float = 0.7):
    global df, cleaned_contents, kw_extractor, fair_ner_pipe
    # Initialize article groups
    article_groups = []
    unclassified_articles = []  # To store articles with no entities/keywords
    tqdm_bar = tqdm(enumerate(zip(cleaned_contents, df.iterrows())), 
                    desc="Grouping articles... (Groups: 0)", 
                    total=len(cleaned_contents))

    for i, (cleaned_content, (row_idx, row)) in tqdm_bar:
        # Cleaned content into text
        cleaned_content.insert(0, row.title)
        cleaned_content = " ".join(cleaned_content)
        # Extract keywords
        keywords = [keyword[0] for keyword in kw_extractor.extract_keywords(cleaned_content)]
        # Extract entities
        raw_entities = fair_ner_pipe(cleaned_content)
        entities = combine_same_entities(_text=cleaned_content, _raw_entities=raw_entities)
        entities = [Entity(raw_entity) for raw_entity in raw_entities]
        
        # Check if article has no entities and no keywords
        if not entities or not keywords:
            unclassified_articles.append(row['id'])
            continue
            
        # Check if the article is relevant to any existing group
        is_grouped = False
        for group in article_groups:
            if group.get_group_relevance_score(entities, keywords, match_threshold=threshold):
                group.add_article(row['id'], entities, keywords)  # Use row['id'] instead of i
                is_grouped = True
                break
        # If not grouped, create a new group
        if not is_grouped:
            new_group = ArticleGroup(identifier=len(article_groups))
            new_group.add_article(row['id'], entities, keywords)  # Use row['id'] instead of i
            article_groups.append(new_group)
            # Update progress bar description with group count
            tqdm_bar.set_description(f"Grouping articles... (Threshold: {threshold}) (Groups: {len(article_groups)})")

    # Create an "Unclassified" group if there are any unclassified articles
    if unclassified_articles:
        unclassified_group = ArticleGroup(identifier=len(article_groups))
        unclassified_group.name = "Unclassified"  # Set a name for the unclassified group
        
        # Add each unclassified article to this group
        for article_idx in unclassified_articles:
            # Adding with empty entities and keywords
            unclassified_group.add_article(article_idx, [], [])
        
        # Add the unclassified group to the end of article_groups
        article_groups.append(unclassified_group)
        tqdm_bar.set_description(f"Grouping articles... (Threshold: {threshold}) (Groups: {len(article_groups)})")
    
    return article_groups

In [ ]:
def export_results(article_groups, filename: str):
    # Export the results to a csv
    group_ids = []
    for group in article_groups:
        group_ids.extend([group.identifier] * len(group.get_group_article_ids()))

    article_ids = []
    for group in article_groups:
        article_ids.extend(group.get_group_article_ids())


    if len(article_ids) != len(group_ids):
        raise ValueError("Mismatch between article_ids and group_ids lengths")

    results_df = pd.DataFrame({"article_id": article_ids, "group_id": group_ids})

    results_df.to_csv(filename, index=False)

In [ ]:
# Trial of different thresholds and exporting results
# 0.1 Threshold
article_groups = group_articles(threshold=0.1)
export_results(article_groups, 'article_groups_0.1t.csv')
# 0.7 Threshold
article_groups = group_articles(threshold=0.7)
export_results(article_groups, 'article_groups_0.7t.csv')
# 0.9 Threshold
article_groups = group_articles(threshold=0.9)
export_results(article_groups, 'article_groups_0.9t.csv')

Grouping articles... (Threshold: 0.1) (Groups: 21): 100%|██████████| 354/354 [16:01<00:00,  2.72s/it]
Grouping articles... (Threshold: 0.7) (Groups: 108): 100%|██████████| 354/354 [14:33<00:00,  2.47s/it]
Grouping articles... (Threshold: 0.9) (Groups: 64):  29%|██▉       | 102/354 [03:12<07:56,  1.89s/it]


KeyboardInterrupt: 

### Evaluate results

In [54]:
# Import evaluation functions
from sklearn.metrics import (
    adjusted_rand_score,
    normalized_mutual_info_score,
    fowlkes_mallows_score,
    rand_score,
    homogeneity_completeness_v_measure
)

In [33]:
gt_df = df[['id', 'event_id']].copy()
gt_df.rename(columns={'id': 'article_id', 'event_id': 'gt_id'}, inplace=True)

In [55]:
# Import result csvs
results_0_1 = pd.read_csv('results/article_groups_0.1t.csv')
results_0_7 = pd.read_csv('results/article_groups_0.7t.csv')
results_0_8 = pd.read_csv('results/article_groups_0.8t.csv')
results_0_9 = pd.read_csv('results/article_groups_0.9t.csv')
results_0_95 = pd.read_csv('results/article_groups_0.95t.csv')
results_0_98 = pd.read_csv('results/article_groups_0.98t.csv')
results_1 = pd.read_csv('results/article_groups_1t.csv')

In [56]:
results_0_1 = results_0_1.merge(gt_df, on='article_id', how='left')
results_0_7 = results_0_7.merge(gt_df, on='article_id', how='left')
results_0_8 = results_0_8.merge(gt_df, on='article_id', how='left')
results_0_9 = results_0_9.merge(gt_df, on='article_id', how='left')
results_0_95 = results_0_95.merge(gt_df, on='article_id', how='left')
results_0_98 = results_0_98.merge(gt_df, on='article_id', how='left')
results_1 = results_1.merge(gt_df, on='article_id', how='left')

In [62]:
# Evaluate each result df and print the result as a single dataframe
def evaluate_results(pred_labels: pd.Series, ground_truth: pd.Series) -> dict:
    """Evaluate the results using various clustering metrics.

    Args:
        pred_labels (pd.Series): Series containing the predicted group IDs.
        ground_truth (pd.Series): Series containing the true group IDs for each article.

    Returns:
        dict: Dictionary containing the evaluation metrics.
    """
    # Use the provided predicted labels instead of a DataFrame column
    predicted_labels = pred_labels.values
    true_labels = ground_truth.values

    # Calculate evaluation metrics
    metrics = {
        "Rand Index": rand_score(true_labels, predicted_labels),
        "Adjusted Rand Index": adjusted_rand_score(true_labels, predicted_labels),
        "Normalized Mutual Information": normalized_mutual_info_score(true_labels, predicted_labels),
        "Fowlkes-Mallows Score": fowlkes_mallows_score(true_labels, predicted_labels),
        # "Homogeneity": homogeneity_completeness_v_measure(true_labels, predicted_labels)[0],
        # "Completeness": homogeneity_completeness_v_measure(true_labels, predicted_labels)[1],
        # "V-Measure": homogeneity_completeness_v_measure(true_labels, predicted_labels)[2]
    }

    return metrics

# Evaluate each result df
results_0_1_metrics = evaluate_results(results_0_1['group_id'], results_0_1['gt_id'])
results_0_7_metrics = evaluate_results(results_0_7['group_id'], results_0_7['gt_id'])
results_0_8_metrics = evaluate_results(results_0_8['group_id'], results_0_8['gt_id'])
results_0_9_metrics = evaluate_results(results_0_9['group_id'], results_0_9['gt_id'])
results_0_95_metrics = evaluate_results(results_0_95['group_id'], results_0_95['gt_id'])
results_0_98_metrics = evaluate_results(results_0_98['group_id'], results_0_98['gt_id'])
results_1_metrics = evaluate_results(results_1['group_id'], results_1['gt_id'])

# Combine all metrics into a single dataframe
result_vars = {
    "0.1": results_0_1_metrics,
    "0.7": results_0_7_metrics,
    "0.8": results_0_8_metrics,
    "0.9": results_0_9_metrics,
    "0.95": results_0_95_metrics,
    "0.98": results_0_98_metrics,
    "1.0": results_1_metrics
}
results_df = pd.DataFrame(result_vars)
# Print the results
results_df

,0.1,0.7,0.8,0.9,0.95,0.98,1.0
Rand Index,0.661705,0.773003,0.930427,0.973576,0.981226,0.979482,0.969847
Adjusted Rand Index,0.022529,0.026626,0.099219,0.260240,0.334125,0.238809,0.211558
Normalized Mutual Information,0.438296,0.631627,0.776134,0.857454,0.877957,0.865890,0.844180
Fowlkes-Mallows Score,0.125061,0.112621,0.171474,0.301468,0.357833,0.256908,0.253860


In [63]:
# Plot with plotly
import plotly.express as px
import plotly.graph_objects as go

# Create a DataFrame for plotting
plot_df = results_df.reset_index().melt(id_vars='index', var_name='Threshold', value_name='Score')

# Create a line plot
fig = px.line(plot_df, x='Threshold', y='Score', color='index', 
              title='Clustering Evaluation Metrics by Threshold',
              labels={'index': 'Metric', 'Score': 'Score'},
              range_y=[0, 1])
# Show the plot
fig.show()